### ***Collect Data***

* we will be importing the data in from FRED and we will be using `Total Deposits` (DPSACBM027NBOG)

### ***Decomposition of personal savings into trend, seasonal and irregular components***

$$S_{t}= trend + seasonal + cyclical + irregularity$$

To be able to make forecast for the data, we must transform it into a stationary time series. This is done by removing the trend and seasonal components. 
- The `trend` component is removed by applying a transformation that dampens the trend. 
- The `seasonal` component is removed by applying a transformation that dampens the seasonal component. 
- The `irregular` component is removed by applying a transformation that dampens the irregular component. 
- The `cyclical` component is removed by applying a transformation that dampens the cyclical component.

Our **GOAL** is to have a clearer signal: Identifying and removing anything but the irregular component from the time series can result in a clearer relationship between input and output variables.



In [ ]:
from fredapi import Fred
import plotly.graph_objects as go #used for data visualization
import pandas as pd

fred = Fred(api_key='0e929d96cfbf2133c9a346a74670f033')
df = pd.DataFrame(fred.get_series('DPSACBW027NBOG')).reset_index()

# renaming columns
df.columns = ['report_date', 'total_deposits']

# only include dates from 2017+ 
df = df[df['report_date'] >= '2017-01-01']

# visualizing 
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))

### ***Trend component***
 Currently form: $$ S_{t} = trend + seasonal + cyclical + irregularity$$
 Goal form: $$ S_{t} = seasonal + cyclical + irregularity$$
To remove the trend from the data, we take a moving average of the data and remove it from the raw data. This will remove the long term pattern from the data and we will be left with the seasonal, cyclical and irregular components.

In [ ]:
""" Method 1 - Moving Average """

# get moving average for past 2 observations (weeks) not including current observation
df['ma'] = df['total_deposits'].shift(1).rolling(window=24).mean()

# remove the first observation since it will be null
df = df.dropna()

# difference between current observation and moving average
df['total_deposits_detrended_ma'] = df['total_deposits'] - df['ma']

# visualizing
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits_detrended_ma'], name='Total Deposits Detrended (ma)'))
fig.add_trace(go.Scatter(x=df['report_date'], y=df['ma'], name='Moving Average'))
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))


In [ ]:
""" Method 2 - Differencing (Random Walk) """

df['total_deposits_detrended_diff'] = df['total_deposits'].diff(periods=1)

# visualizing
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits_detrended_diff'], name='Total Deposits Detrended (diff)'))
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))


This will remove the long term pattern from the data and we will be left with the seasonal, cyclical and irregular components.

Now our data is of form:
$$S_{t}= seasonal + cyclical + irregularity$$

### ***Seasonal component***

 Currently form: $$ S_{t} = trend + seasonal + cyclical + irregularity$$
 Goal form: $$ S_{t} = trend + cyclical + irregularity$$
To remove the seasonal component from the data, we will use statsmodels seasonal_decompose function. This will remove the seasonal component from the data and we will be left with the trend, cyclical and irregular components.

In [ ]:
""" method 1: use statsmodels """

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Assuming you have already loaded your DataFrame df
# df['date'] = pd.to_datetime(df['report_date'])
# df.set_index('date', inplace=True)

result = seasonal_decompose(df['total_deposits'], model='additive', period=12)
# period = 7 if we suspect a weekly cycle
# period = 1 if we suspect a monthly cycle
# period = 4 if we suspect a quarterly cycle
# period = 12 if we suspect an annual cycle

df['seasonal'] = result.seasonal

df['total_deposits_sa'] = df['total_deposits'] - df['seasonal']

# detrended using R.W and then seasonally adjusted
df['total_deposits_sa_detrended_diff'] = df['total_deposits_detrended_diff'] - df['seasonal']

# detrended using moving average and then seasonally adjusted
df['total_deposits_sa_detrended_ma'] = df['total_deposits_detrended_ma'] - df['seasonal']

fig = go.Figure()
fig.update_layout(title='Total Deposits (SA)')
# fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))
fig.add_trace(go.Scatter(x=df['report_date'], y=df['seasonal'], name='Seasonal Component'))
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits_sa'], name='Total Deposits (SA)'))

### ***Visualizations***

In [ ]:
fig = go.Figure()
fig.update_layout(title='Original')
fig.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))
fig.show()

fig2 = go.Figure()
fig2.update_layout(title='Trend')
fig2.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits'], name='Total Deposits'))
fig2.add_trace(go.Scatter(x=df['report_date'], y=df['ma'], name='Moving Average'))
fig2.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits_detrended_diff'], name='Total Deposits Detrended (diff)'))
fig2.show()

fig3 = go.Figure()
fig3.update_layout(title='Seasonal')
fig3.add_trace(go.Scatter(x=df['report_date'], y=df['seasonal'], name='seasonal component'))
fig3.show()

fig4 = go.Figure()
fig4.update_layout(title='Residual')
fig4.add_trace(go.Scatter(x=df['report_date'], y=df['total_deposits_sa_detrended_diff'], name='Total Deposits (SA) Detrended'))
fig4.show()

### ***Cyclical component***

 Currently form: $$ S_{t} = trend + seasonal + cyclical + irregularity$$
 Goal form: $$ S_{t} = trend + seasonal + irregularity$$

### ACF TEST

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

dff = df.dropna()

plot_acf(dff['total_deposits_sa_detrended_diff'], lags=30)

plot_acf(dff['total_deposits_sa_detrended_ma']) 

# how do we get lags from this?

# what model should we use?

bc moving avg we are calculating 24 weeks, so the diff btw each week is going to matter less -> smoooth acf

dff-> only acct for diff between 2 weeks so diff is the only thing that matters -> ratical acf changes